Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (195013, 28, 28) (195013,)
Validation set (9847, 28, 28) (9847,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195013, 784) (195013, 10)
Validation set (9847, 784) (9847, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    #Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    probs = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels)
    loss = tf.reduce_mean(probs + beta * tf.nn.l2_loss(weights))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

In [6]:
num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.954170
Minibatch accuracy: 5.5%
Validation accuracy: 10.4%
Minibatch loss at step 500: 2.724561
Minibatch accuracy: 80.5%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.558977
Minibatch accuracy: 83.6%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 1.342604
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 1.061696
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 0.733435
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.738704
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 3500: 0.615566
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Minibatch loss at step 4000: 0.996096
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 4500: 0.783358
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Test accuracy: 89.1%


We used beta = 0.001 as the regularization factor, and it slightly reduced the test accuracy. Therefore, we run the training and test process for a few values of beta in order to pick the right value.

In [7]:
num_steps = 5000
beta_values = [pow(10.0, i) for i in range(-5,2)]
accuracies = []

for beta_factor in beta_values:
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_factor}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        accuracies.append(accuracy(test_prediction.eval(), test_labels))

In [8]:
max_accuracy = max(accuracies)
max_index = accuracies.index(max_accuracy)
print('The maximum accuracy was ', max_accuracy, ' for the beta value ', beta_values[max_index],'.', sep='')

The maximum accuracy was 89.11 for the beta value 0.001.


Therefore, we keep 0.001 as the regularization factor. Now we add the regularization to the neural net with ReLU hidden layer.

In [9]:
batch_size = 128
num_nodes = 1024

graph2 = tf.Graph()
with graph2.as_default():
    
    #Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    #Variables
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    beta = tf.placeholder(tf.float32)
    
    #Computation
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits = tf.matmul(hidden_layer1, weights2) + biases2
    probs = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels)
    loss = tf.reduce_mean(probs + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights2) + biases2)
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2)

In [10]:
num_steps = 5000

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 623.708557
Minibatch accuracy: 9.4%
Validation accuracy: 26.8%
Minibatch loss at step 500: 193.358978
Minibatch accuracy: 86.7%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 114.433151
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 69.197296
Minibatch accuracy: 82.0%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 41.457825
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.194443
Minibatch accuracy: 85.2%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.501881
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Minibatch loss at step 3500: 9.532675
Minibatch accuracy: 88.3%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 6.261333
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 3.847792
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Test accuracy: 94.3%


The test accuracy has improved. This shows that our model was overfitting before applying regularization.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
num_steps = 5000
num_batches = 5

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 712.426575
Minibatch accuracy: 3.9%
Validation accuracy: 33.3%
Minibatch loss at step 500: 190.579819
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 115.577957
Minibatch accuracy: 100.0%
Validation accuracy: 74.6%
Minibatch loss at step 1500: 70.093185
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 2000: 42.509117
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 25.781807
Minibatch accuracy: 100.0%
Validation accuracy: 75.1%
Minibatch loss at step 3000: 15.639572
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3500: 9.492173
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 4000: 5.769144
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 4500: 3.516703
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 86.7%


With few batches, the minibatch accuracy is 100 percent while the validation and test accuracy have decreased. This indicates that our model is overfitting. 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128
num_nodes = 1024

graph3 = tf.Graph()
with graph3.as_default():
    
    #Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    #Variables
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    beta = tf.placeholder(tf.float32)
    
    #Computation
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    drop = tf.nn.dropout(hidden_layer1, 0.5)
    logits = tf.matmul(drop, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(layer1_valid, weights2) + biases2)
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2)

In [13]:
num_steps = 5000
num_batches = 5

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 533.648560
Minibatch accuracy: 3.9%
Validation accuracy: 26.7%
Minibatch loss at step 500: 0.321836
Minibatch accuracy: 98.4%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 0.444124
Minibatch accuracy: 99.2%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Test accuracy: 87.0%


We replaced the l2 regularization with a dropout layer, and the test accuracy of the model did not change significantly. Thus, our model is still overfitting because of the low number of training batches.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


We train a neural network with three hidden layers and 1024, 512, and 60 nodes, respectively. All weights are initialized using random numbers from a normal distribution with zero mean and sqrt(2/(number of nodes)) standard deviation. There is a dropout layer with 0.5 rate after each fully connected layer in order to prevent the model from overfitting. Tha activation function of all layers is ReLU.

In [14]:
batch_size = 256
num_nodes1 = 1024
num_nodes2 = 512
num_nodes3 = 60

graph4 = tf.Graph()
with graph4.as_default():
    
    #Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    #Variables
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_nodes1],stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([num_nodes1]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes1, num_nodes2],stddev=np.sqrt(2.0 /num_nodes1)))
    biases2 = tf.Variable(tf.zeros([num_nodes2]))
    weights3 = tf.Variable(tf.truncated_normal([num_nodes2, num_nodes3],stddev=np.sqrt(2.0 /num_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_nodes3]))
    weights4 = tf.Variable(tf.truncated_normal([num_nodes3, num_labels],stddev=np.sqrt(2.0 /num_nodes3)))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    global_step = tf.Variable(0)
    beta = tf.placeholder(tf.float32)
    
    #Computation
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    drop1 = tf.nn.dropout(hidden_layer1, 0.5)
    hidden_layer2 = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
    drop2 = tf.nn.dropout(hidden_layer2, 0.5)
    hidden_layer3 = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
    drop3 = tf.nn.dropout(hidden_layer3, 0.5)
    logits = tf.matmul(drop3, weights4) + biases4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels))
    
    #Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.9)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    layer2_valid = tf.nn.relu(tf.matmul(layer1_valid, weights2) + biases2)
    layer3_valid = tf.nn.relu(tf.matmul(layer2_valid, weights3) + biases3)
    valid_prediction = tf.nn.softmax(tf.matmul(layer3_valid, weights4) + biases4)
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    layer2_test = tf.nn.relu(tf.matmul(layer1_test, weights2) + biases2)
    layer3_test = tf.nn.relu(tf.matmul(layer2_test, weights3) + biases3)
    test_prediction = tf.nn.softmax(tf.matmul(layer3_test, weights4) + biases4)
    

In [15]:
num_steps = 5000

with tf.Session(graph=graph4) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.733252
Minibatch accuracy: 10.9%
Validation accuracy: 18.3%
Minibatch loss at step 500: 0.439369
Minibatch accuracy: 85.2%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.573270
Minibatch accuracy: 84.0%
Validation accuracy: 85.8%
Minibatch loss at step 1500: 0.395725
Minibatch accuracy: 87.9%
Validation accuracy: 86.7%
Minibatch loss at step 2000: 0.461807
Minibatch accuracy: 86.3%
Validation accuracy: 87.1%
Minibatch loss at step 2500: 0.598049
Minibatch accuracy: 82.8%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.353835
Minibatch accuracy: 88.3%
Validation accuracy: 87.9%
Minibatch loss at step 3500: 0.384083
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 4000: 0.439359
Minibatch accuracy: 85.2%
Validation accuracy: 88.5%
Minibatch loss at step 4500: 0.364753
Minibatch accuracy: 91.4%
Validation accuracy: 88.6%
Test accuracy: 95.1%


The test accuracy is __95.1__ percent. The performance of the model can still be improved by tuning its hyperparameters, such as number of steps, optimizer, number of layers and nodes, activation function, dropout rate, etc. 